Notebook này dùng ds101 là đủ

Dự kiến sẽ viết thành .py

TODO:
- Viết hàm download
- Xử lí Duplicate (có thể dùng pandas)

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib3
import os
import time
import requests

In [2]:
def getSongLinks(country, tab, start_page=1, end_page=20):
    '''
    Trả về danh sách link tới các bài hát cụ thể để nghe nhạc.
    
    Tham số:
        @country: Thể loại nhạc theo tên quốc gia
        @tab: Tab của trang web: album-2021, album-moi, bai-hat-moi, vua-download
        @start_page: Giới hạn bên trái của trang thể hiện bài hát (def=1)
        @end_page: Giới hạn bên phải của trang thể hiện bài hát (def=20)
    
    Lưu ý:
        Với mỗi tab `vua-download` và `bao-hat-moi`
        sẽ chỉ crawl được trang thuộc [1; 19].
        Trang 19+ không crawl được, đừng để bị số lượng trang lừa ¯_(ツ)_/¯
    
    '''
    driver = webdriver.Chrome("chromedriver_win32/chromedriver")
    URL = "https://www.chiasenhac.vn/mp3/" + country + ".html?tab=" + tab
    arr = np.arange(start_page, end_page)
    save_path = 'data/'
    link_list = []

    for i in arr:
        driver.get(URL + "&page=" + str(i))
        main = driver.find_element_by_class_name('content-current')
        songs = main.find_elements_by_css_selector('.media.align-items-stretch.not')
        for song in songs:
            a = song.find_element_by_tag_name("a")
            link = a.get_attribute("href")
            link_list.append(link)

    driver.close()
    return link_list

In [40]:
def getSongInfo(link_list):
    '''
    Trả về DataFrame danh sách các bài hát:
    1. Title
    2. Artist
    3. Link bài hát
    4. Link .m4a
    5. Link .mp3
    '''
    df_headers = ["Title", "Artist", "Link bài hát", "Link .m4a", "Link .mp3"]
    song_info_list = []
    
    for url in link_list:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        # lấy Title và Artist
        name = soup.find(class_="d-flex justify-content-between mb-3 box1 music-listen-title")
        name = name.h1.text.split(" - ")
        title = name[0]
        artist = name[1]

        # lấy link tải m4a và mp3
        download_div = soup.find(id="pills-download")
        download_items = download_div.find_all("a", class_="download_item", href=True)
        m4a_link = download_items[0]['href']
        mp3_link = download_items[1]['href']

        # Thêm bài hát vào danh sách
        song_info_list.append((title, artist, url, m4a_link, mp3_link))
        
    return pd.DataFrame(song_info_list, columns=df_headers)

## Điều chỉnh thông số tại đây

In [62]:
country = "us-uk"
tab = "bai-hat-moi"
csv_path = "data/{}_{}.csv".format(country,tab)

In [60]:
# Lấy link
link_list = getSongLinks(country, tab)

In [64]:
print("Số bài hát: " + str(len(link_list)))

Số bài hát: 380


In [65]:
# Trích xuất info thanh dataframe
df = getSongInfo(link_list)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         380 non-null    object
 1   Artist        380 non-null    object
 2   Link bài hát  380 non-null    object
 3   Link .m4a     380 non-null    object
 4   Link .mp3     380 non-null    object
dtypes: object(5)
memory usage: 15.0+ KB


In [68]:
# Xóa phần tử trùng theo: Title && Artist
df_new = df.drop_duplicates(subset=["Title","Artist"])

In [69]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 379 entries, 0 to 379
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         379 non-null    object
 1   Artist        379 non-null    object
 2   Link bài hát  379 non-null    object
 3   Link .m4a     379 non-null    object
 4   Link .mp3     379 non-null    object
dtypes: object(5)
memory usage: 17.8+ KB


In [70]:
# Viết info vào trong .csv ('w': overwrite, 'a': append)
df_new.to_csv(csv_path, index=False, mode='w')